In [ ]:
import pandas as pd

In [ ]:
# 数据预处理
# 导入数据并显示前6行数据来检查数据是否正确
df = pd.read_csv('死亡人数.csv',encoding='gbk')
bf = pd.read_csv('出生人数.csv',encoding='gbk')
pd.set_option('display.max_columns',30)
display(df.head(6))
display(bf.head(6))

In [ ]:
# 合并数据集
df = df.set_index(['Country name', 'Year'], drop=True)
bf = bf.set_index(['Country name', 'Year'], drop=True)
bd = pd.merge(df,bf,on=['Country name','Year'])

In [ ]:
# 数据清洗
bd.isnull().sum() # 查看缺失值
for i in bd.columns:
    x = bd[i].mean()# 计算每列的均值
    bd.fillna({i:x}, inplace=True) # 用均值填充缺失值
bd.isnull().sum() # 查看缺失值是否全部处理完毕

In [85]:
# 清洗字符串格式
import dis
import re

from numpy import disp
for i in bd.columns:
    if bd[i].dtype == 'object':
        bd[i] = bd[i].apply(lambda x: re.sub(r'[^\w\s]','',x))

# 显示数据
pd.set_option('display.max_columns',100)
display(bd.head())

Deaths  Deaths of children under the age of 1  \
Country name Year                                                  
Afghanistan  1950  283668                               106003.0   
             1951  282577                               105086.0   
             1952  280803                               105028.0   
             1953  279684                               104949.0   
             1954  280476                               104767.0   

                   Deaths of children under the age of 5  \
Country name Year                                          
Afghanistan  1950                               156547.0   
             1951                               154810.0   
             1952                               153384.0   
             1953                               152620.0   
             1954                               152387.0   

                   Deaths aged 1 to 4 years  Deaths aged 5 to 9 years  \
Country name Year                                                       
Afghanistan  1950                   50544.0                   10043.0   
             1951                   49724.0                   10225.0   
             1952                   48356.0                   10305.0   
             1953                   47671.0                   10321.0   
             1954                   47620.0                   10480.0   

                   Deaths aged 10 to 14 years  Deaths aged 15 to 19 years  \
Country name Year                                                           
Afghanistan  1950                      6549.0                      7752.0   
             1951                      6588.0                      7802.0   
             1952                      6560.0                      7794.0   
             1953                      6540.0                      7787.0   
             1954                      6716.0                      7914.0   

                   Deaths aged 20 to 29 years  Deaths aged 30 to 39 years  \
Country name Year                                                           
Afghanistan  1950                     18036.0                     17161.0   
             1951                     18161.0                     17247.0   
             1952                     18121.0                     17179.0   
             1953                     18077.0                     17114.0   
             1954                     18244.0                     17234.0   

                   Deaths aged 40 to 49 years  Deaths aged 50 to 59 years  \
Country name Year                                                           
Afghanistan  1950                     16389.0                     17605.0   
             1951                     16367.0                     17521.0   
             1952                     16218.0                     17323.0   
             1953                     16081.0                     17139.0   
             1954                     16109.0                     17108.0   

                   Deaths aged 60 to 69 years  Deaths aged 70 to 79 years  \
Country name Year                                                           
Afghanistan  1950                     18715.0                     12380.0   
             1951                     18681.0                     12536.0   
             1952                     18531.0                     12615.0   
             1953                     18401.0                     12700.0   
             1954                     18404.0                     12849.0   

                   Deaths aged 80 to 89 years  Deaths aged 90 to 99 years  \
Country name Year                                                           
Afghanistan  1950                      2434.0                        55.0   
             1951                      2594.0                        49.0   
             1952                      2716.0                        58.0   
             1953                      2828.0                   

In [96]:
# 数据分析
# 总览分析
# 计算全球总出生人数和总死亡人数
total_birth = bd.loc[('World'), 'Births'].astype('int64').sum() # type: ignore
display(total_birth)
total_death = bd.loc[('World'), 'Deaths'].astype('int64').sum() # type: ignore
display(total_death)

# 计算各国的总出生人数和总死亡人数
total_births = bd.loc[:, 'Births'].astype('int64')
display(total_births)


9119585530

3655573526

Country name  Year
Afghanistan   1950    365303
              1951    372040
              1952    378290
              1953    384933
              1954    390412
                       ...  
Zimbabwe      2017    480635
              2018    483783
              2019    484906
              2020    486393
              2021    488807
Name: Births, Length: 18360, dtype: int64